<a href="https://colab.research.google.com/github/Anum79/AI-Q2-learning-resources/blob/master/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First import libraries**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.ma.core import shape
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


**Load data**

In [ ]:
df = pd.read_csv("/content/OANDA_USDCAD, 5 - Def.csv")
df = df.rename(columns={'<DATE>':'date', '<TIME>':'time', '<OPEN>':'open', '<HIGH>':'high', '<LOW>':'low', '<CLOSE>':'close'})

**check head**

In [ ]:
df.head()

,time,open,high,low,close,Action
0,6/12/2022 16:00:00,1.278525,1.278620,1.27828,1.278487,0
1,6/12/2022 16:05:00,1.278506,1.278506,1.27834,1.278395,0
2,6/12/2022 16:10:00,1.278451,1.278451,1.27821,1.278307,0
3,6/12/2022 16:15:00,1.278379,1.278379,1.27826,1.278275,0
4,6/12/2022 16:20:00,1.278327,1.278327,1.27815,1.278225,0


**Change time object in string**

In [ ]:
df['time'] = pd.to_datetime(df['time'])

**Check head again**

In [ ]:
df.head()

,time,open,high,low,close,Action
0,2022-06-12 16:00:00,1.278525,1.278620,1.27828,1.278487,0
1,2022-06-12 16:05:00,1.278506,1.278506,1.27834,1.278395,0
2,2022-06-12 16:10:00,1.278451,1.278451,1.27821,1.278307,0
3,2022-06-12 16:15:00,1.278379,1.278379,1.27826,1.278275,0
4,2022-06-12 16:20:00,1.278327,1.278327,1.27815,1.278225,0


**Values to int64**

In [ ]:
df_unix_sec = pd.to_datetime(df['time']).astype(int)/ 10**9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """Entry point for launching an IPython kernel.


Time info

In [ ]:
df_unix_sec

0        1.655050e+09
1        1.655050e+09
2        1.655050e+09
3        1.655050e+09
4        1.655051e+09
             ...     
10297    1.659350e+09
10298    1.659351e+09
10299    1.659351e+09
10300    1.659351e+09
10301    1.659352e+09
Name: time, Length: 10302, dtype: float64

In [ ]:
df_unix_sec.dtype

dtype('float64')

In [ ]:
df.head()

,time,open,high,low,close,Action
0,2022-06-12 16:00:00,1.278525,1.278620,1.27828,1.278487,0
1,2022-06-12 16:05:00,1.278506,1.278506,1.27834,1.278395,0
2,2022-06-12 16:10:00,1.278451,1.278451,1.27821,1.278307,0
3,2022-06-12 16:15:00,1.278379,1.278379,1.27826,1.278275,0
4,2022-06-12 16:20:00,1.278327,1.278327,1.27815,1.278225,0


**Now showing time and date in seconds**

In [ ]:
df['time'] = df_unix_sec

In [ ]:
df.head()

,time,open,high,low,close,Action
0,1.655050e+09,1.278525,1.278620,1.27828,1.278487,0
1,1.655050e+09,1.278506,1.278506,1.27834,1.278395,0
2,1.655050e+09,1.278451,1.278451,1.27821,1.278307,0
3,1.655050e+09,1.278379,1.278379,1.27826,1.278275,0
4,1.655051e+09,1.278327,1.278327,1.27815,1.278225,0


In [ ]:
df.columns

Index(['time', 'open', 'high', 'low', 'close', 'Action'], dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10302 entries, 0 to 10301
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    10302 non-null  float64
 1   open    10302 non-null  float64
 2   high    10302 non-null  float64
 3   low     10302 non-null  float64
 4   close   10302 non-null  float64
 5   Action  10302 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 483.0 KB


In [ ]:
df.describe()

,time,open,high,low,close,Action
count,1.030200e+04,10302.000000,10302.000000,10302.000000,10302.000000,10302.000000
mean,1.657130e+09,1.292972,1.293287,1.292660,1.292973,0.044457
std,1.246951e+06,0.007515,0.007550,0.007488,0.007519,0.364639
min,1.655050e+09,1.277034,1.277110,1.276730,1.276891,0.000000
25%,1.655996e+09,1.287239,1.287480,1.286960,1.287216,0.000000
50%,1.657115e+09,1.292323,1.292667,1.292012,1.292319,0.000000
75%,1.658233e+09,1.298497,1.298770,1.298240,1.298498,0.000000
max,1.659352e+09,1.321139,1.322360,1.320315,1.321962,4.000000


**Taking comlumn as input**

In [ ]:
X = df.iloc[: , :5].values

**Labels**

In [ ]:
Y = df.iloc[:,-1:].values

In [ ]:
X

array([[1.65504960e+09, 1.27852500e+00, 1.27862000e+00, 1.27828000e+00,
        1.27848750e+00],
       [1.65504990e+09, 1.27850625e+00, 1.27850625e+00, 1.27834000e+00,
        1.27839500e+00],
       [1.65505020e+09, 1.27845063e+00, 1.27845063e+00, 1.27821000e+00,
        1.27830750e+00],
       ...,
       [1.65935100e+09, 1.28270481e+00, 1.28364000e+00, 1.28268000e+00,
        1.28316375e+00],
       [1.65935130e+09, 1.28293428e+00, 1.28388000e+00, 1.28293428e+00,
        1.28347875e+00],
       [1.65935160e+09, 1.28320651e+00, 1.28326000e+00, 1.28302000e+00,
        1.28314625e+00]])

In [ ]:
X.size

51510

In [ ]:
from numpy.ma.core import shape
X.shape

(10302, 5)

In [ ]:
Y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
Y.size

10302

In [ ]:
Y.shape

(10302, 1)

**Splitting data **

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=0)

In [ ]:
x_train.shape

(8241, 5)

In [ ]:
x_test.shape

(2061, 5)

In [ ]:
y_test.shape

(2061, 1)

In [ ]:
y_train.shape

(8241, 1)

**Model Training**

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(random_state=0)
xgb_model.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(objective='multi:softprob')

In [ ]:
x_test[0]

array([1.65687900e+09, 1.28857045e+00, 1.28935000e+00, 1.28857045e+00,
       1.28915500e+00])

In [ ]:
y_test[0]

array([0])

**Now predicting**

In [ ]:
yp = xgb_model.predict(x_test)
yp

array([0, 0, 0, ..., 0, 0, 0])

**Confusion matrix showing accuracy**

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,yp)
cm

array([[2027,    0,    1,    0,    0],
       [  11,    0,    0,    0,    0],
       [   6,    0,    0,    0,    0],
       [   8,    0,    0,    0,    0],
       [   8,    0,    0,    0,    0]])

**Model accuracy**

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,yp) * 100

98.35031538088307